# Python语言特性

## 1 Python的函数参数传递

看两个例子:

In [1]:
a = 1
def fun(a):
    a = 2
fun(a)
print(a)

1


In [2]:
a = []
def fun(a):
    a.append(1)
fun(a)
print(a)

[1]


所有的变量都可以理解是内存中一个对象的“引用”，或者，也可以看似c中void*的感觉。

通过`id`来看引用`a`的内存地址可以比较理解：

In [3]:
a = 1
def fun(a):
    print("func_in",id(a))
    a = 2
    print("re-point",id(a), id(2))
print("func_out",id(a), id(1))
fun(a)
print(a)

func_out 4367176376 4367176376
func_in 4367176376
re-point 4367176408 4367176408
1


注：具体的值在不同电脑上运行时可能不同。

可以看到，在执行完`a = 2`之后，`a`引用中保存的值，即内存地址发生变化，由原来`1`对象的所在的地址变成了`2`这个实体对象的内存地址。

而第2个例子`a`引用保存的内存值就不会发生变化：

In [4]:
a = []
def fun(a):
    print ("func_in",id(a))
    a.append(1)
print("func_out",id(a))
fun(a)
print(a)

func_out 4467128000
func_in 4467128000
[1]


这里记住的是类型是属于对象的，而不是变量。而对象有两种,“可更改”（mutable）与“不可更改”（immutable）对象。在python中，strings, tuples, 和numbers是不可更改的对象，而 list, dict, set 等则是可以修改的对象。(这就是这个问题的重点)

当一个引用传递给函数的时候,函数自动复制一份引用,这个函数里的引用和外边的引用没有半毛关系了.所以第一个例子里函数把引用指向了一个不可变对象,当函数返回的时候,外面的引用没半毛感觉.而第二个例子就不一样了,函数内的引用指向的是可变对象,对它的操作就和定位了指针地址一样,在内存里进行修改.

如果还不明白的话,这里有更好的解释: http://stackoverflow.com/questions/986006/how-do-i-pass-a-variable-by-reference

## 2 Python中的元类(metaclass)

这个非常的不常用,但是像ORM这种复杂的结构还是会需要的,详情请看:http://stackoverflow.com/questions/100003/what-is-a-metaclass-in-python

## 3 @staticmethod和@classmethod

Python其实有3个方法,即静态方法(staticmethod),类方法(classmethod)和实例方法,如下:


In [5]:
# 普通函数
def foo(x):
    print("executing foo(%s)"%(x))

class A(object):
    # 实例方法
    def foo(self,x):
        print("executing foo(%s,%s)"%(self,x))

    # 类方法
    @classmethod
    def class_foo(cls,x):
        print("executing class_foo(%s,%s)"%(cls,x))

    # 静态方法
    @staticmethod
    def static_foo(x):
        print("executing static_foo(%s)"%x)

a=A()

这里先理解下函数参数里面的self和cls.这个self和cls是对类或者实例的绑定,对于一般的函数来说我们可以这么调用`foo(x)`,这个函数就是最常用的,它的工作跟任何东西(类,实例)无关.对于实例方法,我们知道在类里每次定义方法的时候都需要绑定这个实例,就是`foo(self, x)`,为什么要这么做呢?因为实例方法的调用离不开实例,我们需要把实例自己传给函数,调用的时候是这样的`a.foo(x)`(其实是`foo(a, x)`).类方法一样,只不过它传递的是类而不是实例,`A.class_foo(x)`.注意这里的self和cls可以替换别的参数,但是python的约定是这俩,还是不要改的好.

对于静态方法其实和普通的方法一样,不需要对谁进行绑定,唯一的区别是调用的时候需要使用`a.static_foo(x)`或者`A.static_foo(x)`来调用.

| \\      | 实例方法     | 类方法            | 静态方法            |
| :------ | :------- | :------------- | :-------------- |
| a = A() | a.foo(x) | a.class_foo(x) | a.static_foo(x) |
| A       | 不可用      | A.class_foo(x) | A.static_foo(x) |

> 简单概括：
> 
> 实例方法：我要这个 对象（self） 来做事
> 
> 类方法：我要这个 类（cls） 来做事
> 
> 静态方法：我只放在类里，但做事不需要对象也不需要类

更多关于这个问题:
1. http://stackoverflow.com/questions/136097/what-is-the-difference-between-staticmethod-and-classmethod-in-python
2. https://realpython.com/blog/python/instance-class-and-static-methods-demystified/

## 4 类变量和实例变量

**类变量：**

> ​	是可在类的所有实例之间共享的值（也就是说，它们不是单独分配给每个实例的）。例如下例中，num_of_instance 就是类变量，用于跟踪存在着多少个Test 的实例。

**实例变量：**

> 实例化之后，每个实例单独拥有的变量。


In [6]:
class Test(object):  
    num_of_instance = 0  
    def __init__(self, name):  
        self.name = name  
        Test.num_of_instance += 1  
  
if __name__ == '__main__':  
    print(Test.num_of_instance)   # 0
    t1 = Test('jack')  
    print(Test.num_of_instance)   # 1
    t2 = Test('lucy')  
    print(t1.name , t1.num_of_instance)  # jack 2
    print(t2.name , t2.num_of_instance)  # lucy 2

0
1
jack 2
lucy 2


> 补充的例子

In [7]:
class Person:
    name="aaa"

p1=Person()
p2=Person()
p1.name="bbb"
print(p1.name)
print(p2.name)
print(Person.name)

bbb
aaa
aaa


这里`p1.name="bbb"`是实例调用了类变量,这其实和上面第一个问题一样,就是函数传参的问题,`p1.name`一开始是指向的类变量`name="aaa"`,但是在实例的作用域里把类变量的引用改变了,就变成了一个实例变量,self.name不再引用Person的类变量name了.

可以看看下面的例子:

In [8]:
class Person:
    name=[]

p1=Person()
p2=Person()
p1.name.append(1)
print(p1.name)
print(p2.name)
print(Person.name)

[1]
[1]
[1]


当类变量值为可变对象（列表、字典等）时，共享类变量可能会造成意外的结果。

为了避免变量混淆，推荐使用 self 来定义实例变量，使用类名或 cls 来定义类变量。对于可变对象的类变量，可以在类定义时使用深复制来避免共享。

区别：

类变量：所有实例访问的是同一个；

实例变量：每个实例互不影响。

## 5 Python自省

这个也是python彪悍的特性.

自省就是面向对象的语言所写的程序在运行时,所能知道对象的类型.简单一句就是运行时能够获得对象的类型.比如type(),dir(),getattr(),hasattr(),isinstance().


In [9]:
a = [1,2,3]
b = {'a':1,'b':2,'c':3}
c = True
print(type(a),type(b),type(c))
print(isinstance(a,list))

<class 'list'> <class 'dict'> <class 'bool'>
True


常用工具：

type()：查看类型

id()：查看内存地址

dir()：查看对象的属性和方法

hasattr() / getattr() / setattr()：动态访问属性

isinstance() / issubclass()：类型判断

用途：调试、动态操作对象、框架开发等。

## 6 字典推导式

字典推导式是用来快速创建字典的一种表达式格式

```
d = {key: value for (key, value) in iterable.items()}
```

从一个已有字典 iterable 中复制出一个新的字典 d

In [13]:
# 创建一个数字平方的字典
squares = {x: x**2 for x in range(5)}  
print(squares)

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}


In [15]:
# 只保留偶数
even_squares = {x: x**2 for x in range(5) if x % 2 == 0}
print(even_squares)

{0: 0, 2: 4, 4: 16}


## 7 Python中单下划线和双下划线

1. 单前下划线 _var
表示“内部使用”，一种约定，不会阻止访问。

示例：`_internal_var`

2. 双前下划线 __var
触发名称重整（Name Mangling），类内部使用，避免子类重写冲突。

示例：`__private_var` → 实际变为 `_类名__private_var`

3. 双前后下划线 __var__
表示Python的魔法方法或系统定义名，不要自定义。

示例：`__init__, __str__`

4. 单下划线 _
常用于临时变量或表示“我不关心这个值”

实例：`for _ in range(5): pass`

In [16]:
class MyClass():
    def __init__(self):
        self.__superprivate = "Hello"
        self._semiprivate = ", world!"
mc = MyClass()
print(mc.__superprivate)
#__superprivate 实际存储为 _MyClass__superprivate（名称重整）
#_semiprivate 名字不变

AttributeError: 'MyClass' object has no attribute '__superprivate'

In [17]:
print(mc._semiprivate)

, world!


In [18]:
print(mc.__dict__)

{'_MyClass__superprivate': 'Hello', '_semiprivate': ', world!'}


`__foo__`:一种约定,Python内部的名字,用来区别其他用户自定义的命名,以防冲突，就是例如`__init__()`,`__del__()`,`__call__()`这些特殊方法

`_foo`:一种约定,用来指定变量私有.程序员用来指定私有变量的一种方式.不能用from module import * 导入，其他方面和公有一样访问；

`__foo`:这个有真正的意义:解析器用`_classname__foo`来代替这个名字,以区别和其他类相同的命名,它无法直接像公有成员一样随便访问,通过对象名._类名__xxx这样的方式可以访问.

详情见:http://stackoverflow.com/questions/1301346/the-meaning-of-a-single-and-a-double-underscore-before-an-object-name-in-python

或者: http://www.zhihu.com/question/19754941

## 8 字符串格式化:%和.format

.format在许多方面看起来更便利.对于`%`最烦人的是它无法同时传递一个变量和元组.你可能会想下面的代码不会有什么问题:
```
"hi there %s" % name
```

但是,如果name恰好是(1,2,3),它将会抛出一个TypeError异常.为了保证它总是正确的,你必须这样做:

```
"hi there %s" % (name,)   # 提供一个单元素的数组而不是一个参数
```

但是有点丑..format就没有这些问题.你给的第二个问题也是这样,.format好看多了.

你为什么不用它?

* 不知道它(在读这个之前)
* 为了和Python2.5兼容(譬如logging库建议使用`%`([issue #4](https://github.com/taizilongxu/interview_python/issues/4)))

http://stackoverflow.com/questions/5082452/python-string-formatting-vs-format



Python f-string 是**执行字符串格式化的最新Python 语法**。 自Python 3.6 起可用。 Python f 字符串提供了一种更快，更易读，更简明且不易出错的在Python 中格式化字符串的方式。f 字符串的前缀为f，并使用{}括号评估值。 在冒号后指定用于类型，填充或对齐的格式说明符。


1. % 格式化（老方法）

In [20]:
name = "小明"
s = "你好，%s" % name
print(s)  # 你好，小明

你好，小明


但是如果 name 是个元组 (1, 2, 3)：

In [21]:
name = (1, 2, 3)
s = "你好，%s" % name   # ❌ 报错 TypeError

TypeError: not all arguments converted during string formatting

因为 % 会把 (1,2,3) 当作多个占位参数。要避免错误，必须写成：

In [23]:
"你好，%s" % (name,)   # ✅ 注意加`,`变成元组里的“一个元素”

'你好，(1, 2, 3)'

❗这就很麻烦、不直观。

2. .format() 方法（推荐，Python 2.6+）

In [24]:
name = (1, 2, 3)
s = "你好，{}".format(name)  # 👍 自动处理，不报错
print(s)  # 你好，(1, 2, 3)

你好，(1, 2, 3)


优势：
- 更清楚易读
- 参数位置可交换
- 不怕类型，也不怕是元组、list

3. f-string（最推荐，Python 3.6+）

In [26]:
name = "小明"
age = 20
print(f"{name}的年龄是{age}岁") 

小明的年龄是20岁


优势：
- 最快
- 最简洁
- 像写变量一样写进去 {} 中


## 9 迭代器和生成器

这个是stackoverflow里python排名第一的问题,值得一看: http://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do-in-python

这是中文版: http://taizilongxu.gitbooks.io/stackoverflow-about-python/content/1/README.html

这里有个关于生成器的创建问题面试官有考：
问：  将列表生成式中[]改成() 之后数据结构是否改变？ 
答案：是，从列表变为生成器


- 迭代器（Iterator）
    - 定义：
可以使用 next() 获取下一个值的对象。
    - 满足两个条件：
有 __iter__() 方法返回自身
有 __next__() 方法，返回下一个值，直到抛出 StopIteration


In [28]:
lst = [1, 2, 3]
it = iter(lst)  # 得到迭代器对象
print(next(it))  # 1
print(next(it))  # 2

1
2


- 生成器（Generator）
    - 是一种 特殊的迭代器，用来 节省内存、惰性生成值。
    - 有两种创建方式：
      
1️⃣ 使用 yield 关键字的函数（最常见）：

In [31]:
def my_gen():
    yield 1
    yield 2
    yield 3
gen = my_gen()
print(next(gen))
print(next(gen))

1
2


2️⃣ 使用 生成器表达式：


In [33]:
gen = (x * x for x in range(3))
print(next(gen))
print(next(gen))

0
1


In [34]:
# 普通列表：一次性产生所有值
lst = [x * x for x in range(10000)]  # 占很多内存！
# 生成器：一个一个产生值，不占内存
gen = (x * x for x in range(10000))  # 几乎不占内存

通过列表生成式，可以直接创建一个列表。但是，受到内存限制，列表容量肯定是有限的。而且，创建一个包含百万元素的列表，不仅是占用很大的内存空间，如：我们只需要访问前面的几个元素，后面大部分元素所占的空间都是浪费的。因此，没有必要创建完整的列表（节省大量内存空间）。

在Python中，我们可以采用生成器：边循环，边计算的机制—>generator

## 10 `*args` and `**kwargs`

用`*args`和`**kwargs`只是为了方便并没有强制使用它们.

当你不确定你的函数里将要传递多少参数时你可以用`*args`.例如,它可以传递任意数量的参数:

In [37]:
def print_everything(*args):
    for count, thing in enumerate(args):
        print(f'{count}. {thing}')
print_everything('apple', 'banana', 'cabbage')

0. apple
1. banana
2. cabbage


相似的,`**kwargs`允许你使用没有事先定义的参数名:

In [40]:
def table_things(**kwargs):
    for name, value in kwargs.items():
        print(f'{name} = {value}')
table_things(apple = 'fruit', cabbage = 'vegetable')

apple = fruit
cabbage = vegetable


你也可以混着用.命名参数首先获得参数值然后所有的其他参数都传递给`*args`和`**kwargs`.命名参数在列表的最前端.例如:

```
def table_things(titlestring, **kwargs)
```

`*args`和`**kwargs`可以同时在函数的定义中,但是`*args`必须在`**kwargs`前面.

当调用函数时你也可以用`*`和`**`语法.例如:


In [41]:
def print_three_things(a, b, c):
    print(f'a = {a}, b = {b}, c = {c}')
mylist = ['aardvark', 'baboon', 'cat']
print_three_things(*mylist)

a = aardvark, b = baboon, c = cat


就像你看到的一样,它可以传递列表(或者元组)的每一项并把它们解包.注意必须与它们在函数里的参数相吻合.当然,你也可以在函数定义或者函数调用时用*.

http://stackoverflow.com/questions/3394835/args-and-kwargs

In [42]:
# *args：收集多个位置参数为元组
def func(*args):
    print(args)
func(1, 2, 3)

(1, 2, 3)


In [43]:
# **kwargs：收集多个关键字参数为字典
def func(**kwargs):
    print(kwargs)
func(a=1, b=2)

{'a': 1, 'b': 2}


In [44]:
# **kwargs：收集多个关键字参数为字典
def func(x, *args, **kwargs):
    print(x)
    print(args)
    print(kwargs)
func(10, 20, 30, a=1, b=2)

10
(20, 30)
{'a': 1, 'b': 2}


In [45]:
# 也可在调用函数时拆包
def add(x, y):
    print(x + y)

args = (1, 2)
add(*args)  # 拆包为 add(1, 2)

kwargs = {'x': 3, 'y': 4}
add(**kwargs)  # 拆包为 add(x=3, y=4)

3
7


## 11 面向切面编程AOP和装饰器

这个AOP一听起来有点懵,同学面阿里的时候就被问懵了...

装饰器是一个很著名的设计模式，经常被用于有切面需求的场景，较为经典的有插入日志、性能测试、事务处理等。装饰器是解决这类问题的绝佳设计，有了装饰器，我们就可以抽离出大量函数中与函数功能本身无关的雷同代码并继续重用。概括的讲，**装饰器的作用就是为已经存在的对象添加额外的功能。**

这个问题比较大,推荐: http://stackoverflow.com/questions/739654/how-can-i-make-a-chain-of-function-decorators-in-python

中文: http://taizilongxu.gitbooks.io/stackoverflow-about-python/content/3/README.html

### 什么是 AOP？
- AOP（Aspect-Oriented Programming）：把横切关注点（如日志、权限、事务等）从主业务逻辑中分离出来。
- 比如：每个函数运行前都记录日志，但你不想把 print("log") 写一堆 —— 这就是“切面”问题。

Python 装饰器就是实现“切面逻辑”的工具：

In [51]:
def log_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"调用函数{func.__name__}")
        result = func(*args,**kwargs)
        print(f"函数{func.__name__}调用结束")
        return result
    return wrapper
    
@log_decorator
def say_hello(name):
    print(f"Hello,{name}!")
say_hello("Eva")

调用函数say_hello
Hello,Eva!
函数say_hello调用结束


这就是：
- 主业务：say_hello 的功能
- 切面逻辑（非核心）：调用前后加日志
- @log_decorator：你把横切关注点“织入”了函数中➡️这就是简化 AOP

### 总结对比

| 概念         | 内容                            | Python 中的实现     |
|--------------|----------------------------------|----------------------|
| AOP          | 面向切面编程                     | 分离横切逻辑         |
| 切面（Aspect）| 非业务的、重复逻辑（日志、权限）| 装饰器中定义的功能   |
| 织入（Weaving）| 把切面逻辑加进主代码中           | 使用 `@decorator`   |
| Pointcut      | 要拦截的函数/方法                | 被装饰的函数名       |

## 12 鸭子类型

“当看到一只鸟走起来像鸭子、游泳起来像鸭子、叫起来也像鸭子，那么这只鸟就可以被称为鸭子。”

我们并不关心对象是什么类型，到底是不是鸭子，只关心行为。

比如在python中，有很多file-like的东西，比如StringIO,GzipFile,socket。它们有很多相同的方法，我们把它们当作文件使用。

又比如list.extend()方法中,我们并不关心它的参数是不是list,只要它是可迭代的,所以它的参数可以是list/tuple/dict/字符串/生成器等.

鸭子类型在动态语言中经常使用，非常灵活，使得python不像java那样专门去弄一大堆的设计模式。

在 Python 里，对象的类型并不重要，只关注它“有没有这个方法/属性”，而不是它具体属于哪个类

例如：
```
def quack_and_walk(thing):
    thing.quack()
    thing.walk()
```

这个函数不用去判断 thing 是不是 Duck 类，只要它有 quack() 和 walk() 方法 —— 就行！

- 好处：
    - 灵活、简洁
    - 支持多种对象（只要具有期望的行为）
- 不检查类型，可能的问题：
    - 传入了不具备方法的对象，运行时报错（不是静态检查）

## 13 Python中重载

引自知乎:http://www.zhihu.com/question/20053359

函数重载主要是为了解决两个问题。

1. 可变参数类型。
2. 可变参数个数。

另外，一个基本的设计原则是，仅仅当两个函数除了参数类型和参数个数不同以外，其功能是完全相同的，此时才使用函数重载，如果两个函数的功能其实不同，那么不应当使用重载，而应当使用一个名字不同的函数。

好吧，那么对于情况 1 ，函数功能相同，但是参数类型不同，python 如何处理？答案是根本不需要处理，因为 python 可以接受任何类型的参数，如果函数的功能相同，那么不同的参数类型在 python 中很可能是相同的代码，没有必要做成两个不同函数。

那么对于情况 2 ，函数功能相同，但参数个数不同，python 如何处理？大家知道，答案就是缺省参数。对那些缺少的参数设定为缺省参数即可解决问题。因为你假设函数功能相同，那么那些缺少的参数终归是需要用的。

好了，鉴于情况 1 跟 情况 2 都有了解决方案，python 自然就不需要函数重载了。

> Python 不支持传统意义上的函数重载（即同名函数定义多个版本），但是可以通过以下方式实现类似功能：

- 方法一：默认参数
```
def greet(name="Guest"):
    print(f"Hello, {name}!")
```

- 方法二：*args / **kwargs
```
def add(*args):
    return sum(args)
```

- 方法三：类型判断
```
def show(x):
    if isinstance(x, int):
        print("Number:", x)
    elif isinstance(x, str):
        print("String:", x)
```

- 方法四：使用 @singledispatch 实现函数重载（标准库）
```
from functools import singledispatch
@singledispatch
def func(val):
    print("Default:", val)
@func.register(int)
def _(val):
    print("It's an int:", val)
@func.register(str)
def _(val):
    print("It's a str:", val)
func(10)     # It's an int: 10
func("hi")   # It's a str: hi
```

    - @singledispatch：声明一个通用函数
    - @func.register(类型)：给特定类型注册不同的处理函数
    - _：函数名随便取，一般用 _ 表示无关紧要
    - 调用时，根据参数类型自动匹配对应版本

> 注意：定义多个同名函数只会保留最后一个定义：
```
def hello(name):
    print("Hi", name)
def hello():
    print("Hi")  # 之前那个被覆盖
```

## 14 新式类和旧式类

这个面试官问了,我说了老半天,不知道他问的真正意图是什么.

[stackoverflow](http://stackoverflow.com/questions/54867/what-is-the-difference-between-old-style-and-new-style-classes-in-python)

这篇文章很好的介绍了新式类的特性: http://www.cnblogs.com/btchenguang/archive/2012/09/17/2689146.html

新式类很早在2.2就出现了,所以旧式类完全是兼容的问题,Python3里的类全部都是新式类.这里有一个MRO问题可以了解下(新式类继承是根据C3算法,旧式类是深度优先),<Python核心编程>里讲的也很多.

> 一个旧式类的深度优先的例子

In [53]:
class A():
    def foo1(self):
        print("A")
class B(A):
    def foo2(self):
        pass
class C(A):
    def foo1(self):
        print("C")
class D(B, C):
    pass

d = D()
d.foo1()

C



**按照经典类的查找顺序`从左到右深度优先`的规则，在访问`d.foo1()`的时候,D这个类是没有的..那么往上查找,先找到B,里面没有,深度优先,访问A,找到了foo1(),所以这时候调用的是A的foo1()，从而导致C重写的foo1()被绕过**

以上为旧式类的行为，在python3新式类中，使用C3 MRO(方法解析顺序)，不会出现C呗绕过的情况

## 15 `__new__`和`__init__`的区别

这个`__new__`确实很少见到,先做了解吧.

1. `__new__`是一个静态方法,而`__init__`是一个实例方法.
2. `__new__`方法会返回一个创建的实例,而`__init__`什么都不返回.
3. 只有在`__new__`返回一个cls的实例时后面的`__init__`才能被调用.
4. 当创建一个新实例时调用`__new__`,初始化一个实例时用`__init__`.


| 方法      | 作用                      | 调用时机             |
|-----------|---------------------------|-----------------------|
| `__new__` | 创建实例：**返回一个对象** | 类实例化开始时最先调用 |
| `__init__`| 初始化实例：**设置属性等** | 在 `__new__` 创建对象后调用 |


[stackoverflow](http://stackoverflow.com/questions/674304/pythons-use-of-new-and-init)

ps: `__metaclass__`是创建类时起作用.所以我们可以分别使用`__metaclass__`,`__new__`和`__init__`来分别在类创建,实例创建和实例初始化的时候做一些小手脚.


### 调用流程：

```
obj = MyClass()
```

等价于：

```
obj = MyClass.__new__(MyClass)
MyClass.__init__(obj)
```

## 16 单例模式

> ​	单例模式是一种常用的软件设计模式。在它的核心结构中只包含一个被称为单例类的特殊类。通过单例模式可以保证系统中一个类只有一个实例而且该实例易于外界访问，从而方便对实例个数的控制并节约系统资源。如果希望在系统中某个类的对象只能存在一个，单例模式是最好的解决方案。
>
> `__new__()`在`__init__()`之前被调用，用于生成实例对象。利用这个方法和类的属性的特点可以实现设计模式的单例模式。单例模式是指创建唯一对象，单例模式设计的类只能实例
**这个绝对常考啊.绝对要记住1~2个方法,当时面试官是让手写的.**

### 1 使用`__new__`方法

In [54]:
class Singleton(object):
    def __new__(cls, *args, **kw):
        if not hasattr(cls, '_instance'):
            orig = super(Singleton, cls)
            cls._instance = orig.__new__(cls, *args, **kw)
        return cls._instance

class MyClass(Singleton):
    a = 1

# 1. Singleton 是一个"只创建一次对象"的基类。
# 2. __new__ 是控制“创建对象”的方法。

### 2 共享属性

创建实例时把所有实例的`__dict__`指向同一个字典,这样它们具有相同的属性和方法.

In [58]:
class Borg(object):
    _state = {}
    def __new__(cls, *args, **kw):
        ob = super(Borg, cls).__new__(cls, *args, **kw)
        ob.__dict__ = cls._state
        return ob

class MyClass2(Borg):
    a = 1

x = MyClass2()
y = MyClass2()
x.a = 100
print(y.a)  # 100，说明状态被共享了
print(x is y)      # False，不是同一个对象
print(x.__dict__)  # {'a': 100}

100
False
{'a': 100}


### 3 装饰器版本

In [60]:
def singleton(cls):
    instances = {}
    def getinstance(*args, **kw):
        if cls not in instances:
            instances[cls] = cls(*args, **kw)
        return instances[cls]
    return getinstance

@singleton
class MyClass:
  ...

# 等价于
# MyClass = singleton(MyClass)

# 使用效果：
# a = MyClass()
# b = MyClass()
# print(a is b)  # True，同一个实例

### 4 import方法

作为python的模块是天然的单例模式

In [63]:
# mysingleton.py
class My_Singleton(object):
    def foo(self):
        pass

my_singleton = My_Singleton()

# to use
# from mysingleton import my_singleton
my_singleton.foo()

## 17 Python中的作用域

Python 中，一个变量的作用域总是由在代码中被赋值的地方所决定的。

当 Python 遇到一个变量的话他会按照这样的顺序进行搜索：

本地作用域（Local）→当前作用域被嵌入的本地作用域（Enclosing locals）→全局/模块作用域（Global）→内置作用域（Built-in）

即LEGB原则

In [66]:
x = 'global'
def outer():
    x = 'enclosing'
    def inner():
        x = 'local'
        print(x)
    inner()
outer()

local


## 18 GIL线程全局锁

线程全局锁(Global Interpreter Lock),即Python为了保证线程安全而采取的独立线程运行的限制,说白了就是一个核只能在同一时间运行一个线程.**对于io密集型任务，python的多线程起到作用，但对于cpu密集型任务，python的多线程几乎占不到任何优势，还有可能因为争夺资源而变慢。**

见[Python 最难的问题](http://www.oschina.net/translate/pythons-hardest-problem)

解决办法就是多进程和下面的协程(协程也只是单CPU,但是能减小切换代价提升性能).

## 19 协程

简单点说协程是进程和线程的升级版,进程和线程都面临着内核态和用户态的切换问题而耗费许多切换时间,而协程就是用户自己控制切换的时机,不再需要陷入系统的内核态.

Python里最常见的yield就是协程的思想!可以查看第九个问题.

## 20 闭包

闭包(closure)是函数式编程的重要的语法结构。闭包也是一种组织代码的结构，它同样提高了代码的可重复使用性。

当一个内嵌函数引用其外部作作用域的变量,我们就会得到一个闭包. 总结一下,创建一个闭包必须满足以下几点:

1. 必须有一个内嵌函数
2. 内嵌函数必须引用外部函数中的变量
3. 外部函数的返回值必须是内嵌函数

感觉闭包还是有难度的,几句话是说不明白的,还是查查相关资料.

重点是函数运行后并不会被撤销,就像16题的instance字典一样,当函数运行完后,instance并不被销毁,而是继续留在内存空间里.这个功能类似类里的类变量,只不过迁移到了函数上.

闭包就像个空心球一样,你知道外面和里面,但你不知道中间是什么样.

In [ ]:
def count_time_wrapper(func):
    def improved_func():
        start_time = time.time()
        func()
        end_time = time.time()
        print(f"it takes {end_time - start_time}s to find all the olds")

    return improved_func
    # 闭包本质上是一个函数
    # 闭包函数的传入参数和返回值都是函数
    # 闭包函数的返回值函数是对传入函数进行增强的函数